<a href="https://colab.research.google.com/github/shantanu2383/Jegadeesh_Titman-1993/blob/main/Jegadeesh_%26_Titman_(1993).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=a73359ac65e66bf21f350dc2e677f2fda24ff827d33c09f1f1d3aee371e1cd6c
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [2]:
#Import CRSP data
filepath="/content/gdrive/MyDrive/Jegadeesh/"
file="CRSPMONTHLY.csv"
crsp=pd.read_csv(filepath + file)

crsp['date'] = pd.to_datetime(crsp['date'], format="%Y%m%d")
crsp=crsp[("1990-01-01"<=crsp['date']) & (crsp['date']< "2022-12-01")]


#parse the relevant variables

crsp_parsed=crsp[['PERMNO', 'date', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']]

crsp=crsp_parsed

#convert variables

'''
VARIABLE DICTIONARY:
PERMNO: security identifier
date: month identifier
RET: return
SHROUT: shares outstanding
ALTPRC: last traded price in a month
EXCHCD: exchange code
SHRCD: share code
SICCD: industry code
DLRET: delisting return
DLSTCD: delisting code
'''

x=['PERMNO', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']

for x in x:
  crsp[x]=pd.to_numeric(crsp[x], errors='coerce')

#convert returns to percent:
crsp['RET']*=100
crsp['DLRET']*=100


#add market cap column
crsp['mkt_cap']=abs(crsp['SHROUT'] * crsp['ALTPRC'])/1000
crsp['mkt_cap'].replace(0, np.NaN, inplace=True)

#rename variables to lower case
crsp= crsp.rename(columns=str.lower)


#filter for relevant exchanges
exchange_mapping = {
    1: 'NYSE', 31: 'NYSE',
    2: 'AMEX', 32: 'AMEX',
    3: 'NASDAQ', 33: 'NASDAQ'
}
#create dictionary to map values of the 'exchcd' column



crsp['exchange'] = np.select(
    [crsp['exchcd'].isin(exchange_mapping.keys()), crsp['exchcd'].notnull()],
    [crsp['exchcd'].map(exchange_mapping), "Other"],
    default='Other')

#only keep NYSE, AMEX  stocks
crsp=crsp[(crsp['exchange'] == "NYSE") | (crsp['exchange']=="AMEX")]

#here we modify the code to avoid looping over the entire dataframe
#we do this by creating masks which are a boolean array of true/false values
#we then use the .loc function to perform vectorised operations
crsp['ret_adj'] = crsp['ret']

mask1 = pd.isnull(crsp['dlstcd'])
crsp.loc[mask1, 'ret_adj'] = crsp.loc[mask1, 'ret']

mask2 = pd.notnull(crsp['dlstcd']) & pd.notnull(crsp['dlret'])
crsp.loc[mask2, 'ret_adj'] = crsp.loc[mask2, 'dlret']

mask3 = ((551 <= crsp['dlstcd']) & (crsp['dlstcd'] <= 574)) | (crsp['dlstcd'].isin([500, 520, 580, 584]))
crsp.loc[mask3, 'ret_adj'] = -30

crsp.loc[~(mask1 | mask2 | mask3), 'ret_adj'] = -100

crsp.drop(['shrcd', 'dlret', 'dlstcd'], inplace=True, axis=1)


#export clean crsp file

crsp= crsp.sort_values(['permno', 'date'])

crsp.set_index('date')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-2-d98fb4977e83>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['ret_adj'] = crsp['ret']
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], val

,permno,ret,shrout,altprc,exchcd,siccd,mkt_cap,exchange,ret_adj
date,,,,,,,,,
2009-12-31,10001,16.2621,4361.0,10.3000,2.0,4925.0,44.918300,AMEX,16.2621
2010-01-29,10001,-1.8932,4361.0,10.0600,2.0,4925.0,43.871660,AMEX,-1.8932
2010-02-26,10001,-0.0656,4361.0,10.0084,2.0,4925.0,43.646632,AMEX,-0.0656
2010-03-31,10001,2.0643,4361.0,10.1700,2.0,4925.0,44.351370,AMEX,2.0643
2010-04-30,10001,12.4385,6070.0,11.3900,2.0,4925.0,69.137300,AMEX,12.4385
...,...,...,...,...,...,...,...,...,...
2021-11-30,93427,15.1667,37018.0,110.5600,1.0,3827.0,4092.710080,NYSE,15.1667
2021-12-31,93427,7.1545,37018.0,118.4700,1.0,3827.0,4385.522460,NYSE,7.1545
2022-01-31,93427,-4.4821,37018.0,113.1600,1.0,3827.0,4188.956880,NYSE,-4.4821


Get lagged returns

In [3]:
crsp['ret_adj']/=100

df=crsp

df['prev_month_return'] = df.groupby('permno')['ret_adj'].shift(1) + 1
df['prev_2month_return'] = df.groupby('permno')['ret_adj'].shift(2) +1 
df['prev_3month_return'] = df.groupby('permno')['ret_adj'].shift(3) +1

df['prev_4month_return'] = df.groupby('permno')['ret_adj'].shift(4) + 1
df['prev_5month_return'] = df.groupby('permno')['ret_adj'].shift(5) +1 
df['prev_6month_return'] = df.groupby('permno')['ret_adj'].shift(6) +1

df['prev_7month_return'] = df.groupby('permno')['ret_adj'].shift(7) + 1
df['prev_8month_return'] = df.groupby('permno')['ret_adj'].shift(8) +1 
df['prev_9month_return'] = df.groupby('permno')['ret_adj'].shift(9) +1

df['prev_10month_return'] = df.groupby('permno')['ret_adj'].shift(10) + 1
df['prev_11month_return'] = df.groupby('permno')['ret_adj'].shift(11) +1 
df['prev_12month_return'] = df.groupby('permno')['ret_adj'].shift(12) +1

#cumulate 3 month returns

df['3_month_return']=df['prev_month_return'] * df['prev_2month_return'] * df['prev_3month_return'] -1

#cumulate 6 month returns

df['6_month_return']=df['prev_month_return'] * df['prev_2month_return'] * df['prev_3month_return'] * df['prev_4month_return'] * df['prev_5month_return'] * df['prev_6month_return'] -1

#cumulate 9 month returns

df['9_month_return']=df['prev_month_return'] * df['prev_2month_return'] * df['prev_3month_return'] * df['prev_4month_return'] * df['prev_5month_return'] * df['prev_6month_return'] * df['prev_7month_return'] * df['prev_8month_return'] * df['prev_9month_return']-1

#cumulate 12 month returns

df['12_month_return']=df['prev_month_return'] * df['prev_2month_return'] * df['prev_3month_return'] * df['prev_4month_return'] * df['prev_5month_return'] * df['prev_6month_return'] * df['prev_7month_return'] * df['prev_8month_return'] * df['prev_9month_return']* df['prev_10month_return'] * df['prev_11month_return'] * df['prev_12month_return']-1

Get returns in following months to calculate portfolio returns

In [4]:
df['month_year']=df['date'].dt.to_period("M")
df.drop(['prev_month_return', 'prev_2month_return', 'prev_3month_return', 'prev_4month_return', 'prev_5month_return', 'prev_6month_return', 'prev_7month_return', 'prev_8month_return', 'prev_9month_return', 'prev_10month_return', 'prev_11month_return', 'prev_12month_return' ], axis=1, inplace=True)
df['ret_adj']+=1

#get returns in the following 6 months

df['t+1_return'] = df.groupby('permno')['ret_adj'].shift(-1) 
df['t+2_return'] = df.groupby('permno')['ret_adj'].shift(-2) 
df['t+3_return'] = df.groupby('permno')['ret_adj'].shift(-3)

df['t+4_return'] = df.groupby('permno')['ret_adj'].shift(-4) 
df['t+5_return'] = df.groupby('permno')['ret_adj'].shift(-5) 
df['t+6_return'] = df.groupby('permno')['ret_adj'].shift(-6)

df['t+7_return'] = df.groupby('permno')['ret_adj'].shift(-7) 
df['t+8_return'] = df.groupby('permno')['ret_adj'].shift(-8) 
df['t+9_return'] = df.groupby('permno')['ret_adj'].shift(-9)

df['t+10_return'] = df.groupby('permno')['ret_adj'].shift(-10) 
df['t+11_return'] = df.groupby('permno')['ret_adj'].shift(-11) 
df['t+12_return'] = df.groupby('permno')['ret_adj'].shift(-12)


# Table 1

K=3

In [6]:
from dateutil.relativedelta import relativedelta
unique_dates=crsp['month_year'].unique()

unique_dates = [x for x in unique_dates if x>(pd.to_datetime("1992-12",  format="%Y-%m").to_period("M"))]
table_1_k_3=pd.DataFrame(columns=['J', 'Portfolio', 'Return',  't'])


for j in [3,6,9,12]:
  for portfolio in [0, 9]:

    final_df=pd.DataFrame(columns=['month_year', 'average_across_3_portfolios', 'p1', 'p2', 'p3', 'p1w', 'p2w', 'p3w', 'wtd_average'])

    for item in unique_dates:
      current_month=str(item)
      current_month=pd.to_datetime(current_month)
      
      #lets consider the first portfolio which was constructed 3 months ago
      current_month=str(current_month)
      current_month=pd.to_datetime(current_month)
      p1_formation_month=(pd.to_datetime((current_month - relativedelta(months=3)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")

      #get stocks for month in which portfolio was formed
      crsp_temp=df[df['month_year']==p1_formation_month]
      crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False)
      crsp_temp_p1=crsp_temp[crsp_temp['portfolio']==portfolio]

    
      #lets now consider the second portfolio whic was constructed 2 months ago
      current_month=str(current_month)
      current_month=pd.to_datetime(current_month)
      p2_formation_month=(pd.to_datetime((current_month - relativedelta(months=2)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")
      p2_formation_month

      #get stocks for month in which portfolio was formed
      crsp_temp=df[df['month_year']==p2_formation_month]
      crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False)
      crsp_temp_p2=crsp_temp[crsp_temp['portfolio']==portfolio]

    
      #lets now consider the third portfolio whic was constructed 1 months ago
      current_month=str(current_month)
      current_month=pd.to_datetime(current_month)
      p3_formation_month=(pd.to_datetime((current_month - relativedelta(months=1)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")


      #get stocks for month in which portfolio was formed
      crsp_temp=df[df['month_year']==p3_formation_month]
      crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False)
      crsp_temp_p3=crsp_temp[crsp_temp['portfolio']==portfolio]

      
      #rebalancing of portfolios- remove any securities that appear across multiple portfolios
      '''
      p1_securities=crsp_temp_p1['permno'].tolist()

      crsp_temp_p2=crsp_temp_p2[~crsp_temp_p2['permno'].isin(p1_securities)]
      
      crsp_temp_p3=crsp_temp_p3[~crsp_temp_p3['permno'].isin(p1_securities) & ~crsp_temp_p3['permno'].isin(crsp_temp_p2['permno'])]
      '''
      
      cols_to_average = ['ret_adj', 't+1_return', 't+2_return']
      averages = crsp_temp_p1[cols_to_average].mean(axis=0)
      p1_return=(averages.prod())**(1/3)
      p1_return
      
      cols_to_average = ['ret_adj', 't+1_return']
      averages = crsp_temp_p2[cols_to_average].mean(axis=0)
      p2_return=(averages.prod())**(1/2)

      
      
      cols_to_average = ['ret_adj']
      averages = crsp_temp_p3[cols_to_average].mean(axis=0)
      p3_return=(averages.prod())**(1)

      pct=crsp['ret_adj'].quantile(0.95)
      #crsp['ret_adj']=crsp['ret_adj'].clip(upper=pct)
      
      #den=len(crsp_temp_p1)+len(crsp_temp_p2)+len(crsp_temp_p3)
      final_df=final_df.append({'month_year': str(item), 'p1': p1_return, 'p2': p2_return, 'p3': p3_return}, ignore_index=True)
      
      
    
   
    
    final_df['average_across_3_portfolios'] = final_df['average_across_3_portfolios'].mean()
    #final_df['wtd_average']=final_df['p1'] * final_df['p1w'] + final_df['p2'] * final_df['p2w'] + final_df['p3'] * final_df['p3w']
    import scipy.stats
   

    final_df['average_across_3_portfolios'] = final_df[['p1', 'p2', 'p3']].mean(axis=1)
    #gmean_ret=scipy.stats.gmean(final_df['average_across_3_portfolios'])
    final_df['average_across_3_portfolios']-=1
    avg_ret = final_df['average_across_3_portfolios'].mean()

    std_error = np.std(final_df['average_across_3_portfolios'], ddof=1)/np.sqrt(len(final_df))
    
    t_stat = (avg_ret ) / std_error
   

    table_1_k_3=table_1_k_3.append({'J': j, 'Portfolio': portfolio, 'Return': avg_ret, 't': t_stat}, ignore_index=True)
    print(j)


<ipython-input-6-1455c97b1afd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False)
<ipython-input-6-1455c97b1afd>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False)
<ipython-input-6-1455c97b1afd>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

3
3
6
6
9
9
12
12


In [7]:
table_1_k_3

,J,Portfolio,Return,t
0,3.0,0.0,0.008093,2.410653
1,3.0,9.0,0.012019,5.475124
2,6.0,0.0,0.006592,1.841732
3,6.0,9.0,0.013914,6.411747
4,9.0,0.0,0.006444,1.720394
5,9.0,9.0,0.014719,7.113392
6,12.0,0.0,0.006299,1.675297
7,12.0,9.0,0.013819,6.715688


k=6

In [8]:
from dateutil.relativedelta import relativedelta
unique_dates=crsp['month_year'].unique()

unique_dates = [x for x in unique_dates if x>(pd.to_datetime("1992-12",  format="%Y-%m").to_period("M"))]
table_1_k_6=pd.DataFrame(columns=['J', 'Portfolio', 'Return', 't'])


for j in [3,6,9,12]:
  for portfolio in [0, 9]:

    final_df=pd.DataFrame(columns=['month_year', 'average_across_6_portfolios', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6'])

    for item in unique_dates:
      current_month=str(item)
      current_month=pd.to_datetime(current_month)
      
      for n in [6,5,4,3,2,1]:
        #lets consider the  portfolio which was constructed n months ago
        current_month=str(current_month)
        current_month=pd.to_datetime(current_month)
        p1_formation_month=(pd.to_datetime((current_month - relativedelta(months=n)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")

        #get stocks for month in which portfolio was formed
        crsp_temp=df[df['month_year']==p1_formation_month]
        crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False, duplicates='drop')
        locals()['crsp_temp_p'  + str(n)]=crsp_temp[crsp_temp['portfolio']==portfolio]
      
      #rebalancing of portfolios- remove any securities that appear across multiple portfolios
      '''
      p6_securities=crsp_temp_p6['permno'].tolist()

      crsp_temp_p5=crsp_temp_p5[~crsp_temp_p5['permno'].isin(p6_securities)]
      
      crsp_temp_p4 = crsp_temp_p4[~crsp_temp_p4['permno'].isin(p6_securities) & ~crsp_temp_p4['permno'].isin(crsp_temp_p5['permno'])]
      
      crsp_temp_p3 = crsp_temp_p3[~crsp_temp_p3['permno'].isin(p6_securities) & ~crsp_temp_p3['permno'].isin(crsp_temp_p5['permno']) & ~crsp_temp_p3['permno'].isin(crsp_temp_p4['permno'])]

      crsp_temp_p2 = crsp_temp_p2[~crsp_temp_p2['permno'].isin(p6_securities) & ~crsp_temp_p2['permno'].isin(crsp_temp_p5['permno']) & ~crsp_temp_p2['permno'].isin(crsp_temp_p4['permno']) & ~crsp_temp_p2['permno'].isin(crsp_temp_p3['permno'])]

      crsp_temp_p1 = crsp_temp_p1[~crsp_temp_p1['permno'].isin(p6_securities) & ~crsp_temp_p1['permno'].isin(crsp_temp_p5['permno']) & ~crsp_temp_p1['permno'].isin(crsp_temp_p4['permno']) & ~crsp_temp_p1['permno'].isin(crsp_temp_p3['permno']) & ~crsp_temp_p1['permno'].isin(crsp_temp_p2['permno'])]
      '''

      #return on p6 (formed 6 months ago)
      cols_to_average = ['ret_adj', 't+1_return', 't+2_return', 't+3_return', 't+4_return', 't+5_return' ]
      averages = crsp_temp_p6[cols_to_average].mean(axis=0)
      p6_return=(averages.prod())**(1/6)
      
      #return on p5 (formed 5 months ago)
      cols_to_average = ['ret_adj', 't+1_return', 't+2_return', 't+3_return', 't+4_return']
      averages = crsp_temp_p5[cols_to_average].mean(axis=0)
      p5_return=(averages.prod())**(1/5)

      #return on p4 (formed 4 months ago)
      cols_to_average = ['ret_adj', 't+1_return', 't+2_return', 't+3_return']
      averages = crsp_temp_p4[cols_to_average].mean(axis=0)
      p4_return=(averages.prod())**(1/4)

      #return on p3 (formed 3 months ago)
      cols_to_average = ['ret_adj', 't+1_return', 't+2_return']
      averages = crsp_temp_p3[cols_to_average].mean(axis=0)
      p3_return=(averages.prod())**(1/3)
      
      #return on p2 (formed 2 months ago)
      cols_to_average = ['ret_adj', 't+1_return']
      averages = crsp_temp_p2[cols_to_average].mean(axis=0)
      p2_return=(averages.prod())**(1/2)

      #return on p1 (formed 1 months ago)
      cols_to_average = ['ret_adj']
      averages = crsp_temp_p1[cols_to_average].mean(axis=0)
      p1_return=(averages.prod())**(1)
      
      final_df=final_df.append({'month_year': str(item), 'p1': p1_return, 'p2': p2_return, 'p3': p3_return, 'p4': p4_return, 'p5': p5_return, 'p6': p6_return }, ignore_index=True)
      
      
    
   
    
    final_df['average_across_6_portfolios'] = final_df[['p1', 'p2', 'p3', 'p4', 'p5', 'p6']].mean(axis=1)
    
    
    avg_ret = final_df['average_across_6_portfolios'].mean()
    gmean_ret=scipy.stats.gmean(final_df['average_across_6_portfolios'])
    std_error = np.std(final_df['average_across_6_portfolios'], ddof=1)/np.sqrt(len(final_df))
    avg_ret=avg_ret-1
    t_stat = (avg_ret ) / std_error
  

    table_1_k_6=table_1_k_6.append({'J': j, 'Portfolio': portfolio, 'Return': avg_ret,  't': t_stat}, ignore_index=True)
    print(j)

<ipython-input-8-4affe865de6f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False, duplicates='drop')


3
3
6
6
9
9
12
12


In [9]:
table_1_k_6

,J,Portfolio,Return,t
0,3.0,0.0,0.006794,2.735417
1,3.0,9.0,0.011858,6.824856
2,6.0,0.0,0.005523,2.061192
3,6.0,9.0,0.013683,8.096703
4,9.0,0.0,0.005465,1.931012
5,9.0,9.0,0.014205,8.736281
6,12.0,0.0,0.005793,2.017725
7,12.0,9.0,0.012875,8.055831


k=9

In [ ]:

table_1_k_9=pd.DataFrame(columns=['J', 'Portfolio', 'Return', 't'])


for j in [3,6,9,12]:
  for portfolio in [0, 9]:

    final_df=pd.DataFrame(columns=['month_year', 'average_across_9_portfolios', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9'])

    for item in unique_dates:
      current_month=str(item)
      current_month=pd.to_datetime(current_month)
      
      cols_to_average=['ret_adj', 't+1_return', 't+2_return', 't+3_return', 't+4_return', 't+5_return', 't+6_return', 't+7_return', 't+8_return' ]
      
      for n in [9,8,7,6,5,4,3,2,1]:
        #lets consider the  portfolio which was constructed n months ago
        current_month=str(current_month)
        current_month=pd.to_datetime(current_month)
        p1_formation_month=(pd.to_datetime((current_month - relativedelta(months=n)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")

        #get stocks for month in which portfolio was formed
        crsp_temp=df[df['month_year']==p1_formation_month]
        crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False, duplicates='drop')
        crsp_temp=crsp_temp[crsp_temp['portfolio']==portfolio]

        averages = crsp_temp[cols_to_average].mean(axis=0)
        locals()['p' + str(n)+ "_return"]=(averages.prod())**(1/n)
        cols_to_average=cols_to_average[:-1]
        
      
      final_df=final_df.append({'month_year': str(item), 'p1': p1_return, 'p2': p2_return, 'p3': p3_return, 'p4': p4_return, 'p5': p5_return, 'p6': p6_return, 'p7': p7_return, 'p8': p8_return, 'p9': p9_return }, ignore_index=True)
      
      
    
   
    
    final_df['average_across_9_portfolios'] = final_df[['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']].mean(axis=1)
    print(final_df)
    
    avg_ret = final_df['average_across_9_portfolios'].mean()
    #gmean_ret=scipy.stats.gmean(final_df['average_across_6_portfolios'])
    std_error = np.std(final_df['average_across_9_portfolios'], ddof=1)/np.sqrt(len(final_df))
    avg_ret=avg_ret-1
    t_stat = (avg_ret ) / std_error
  

    table_1_k_9=table_1_k_9.append({'J': j, 'Portfolio': portfolio, 'Return': avg_ret,  't': t_stat}, ignore_index=True)
    print(j)

    

In [11]:
table_1_k_9

,J,Portfolio,Return,t
0,3.0,0.0,0.006330,3.080987
1,3.0,9.0,0.011766,8.033856
2,6.0,0.0,0.005233,2.353521
3,6.0,9.0,0.013397,9.407677
4,9.0,0.0,0.005372,2.274773
5,9.0,9.0,0.013555,9.863494
6,12.0,0.0,0.005865,2.432373
7,12.0,9.0,0.012170,9.037248


k=12

In [21]:

table_1_k_12=pd.DataFrame(columns=['J', 'Portfolio', 'Return', 't'])


for j in [3,6,9,12]:
  for portfolio in [0, 9]:

    final_df=pd.DataFrame(columns=['month_year', 'average_across_9_portfolios', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12'])

    for item in unique_dates:
      current_month=str(item)
      current_month=pd.to_datetime(current_month)
      
      cols_to_average=['ret_adj', 't+1_return', 't+2_return', 't+3_return', 't+4_return', 't+5_return', 't+6_return', 't+7_return', 't+8_return', 't+9_return', 't+10_return', 't+11_return' ]
      
      for n in [12,10,11,9,8,7,6,5,4,3,2,1]:
        #lets consider the  portfolio which was constructed n months ago
        current_month=str(current_month)
        current_month=pd.to_datetime(current_month)
        p1_formation_month=(pd.to_datetime((current_month - relativedelta(months=n)).strftime("%Y-%m"), format="%Y-%m")).to_period("M")

        #get stocks for month in which portfolio was formed
        crsp_temp=df[df['month_year']==p1_formation_month]
        crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False, duplicates='drop')
        crsp_temp=crsp_temp[crsp_temp['portfolio']==portfolio]

        averages = crsp_temp[cols_to_average].mean(axis=0)
        locals()['p' + str(n)+ "_return"]=(averages.prod())**(1/n)
        cols_to_average=cols_to_average[:-1]
        
      
      final_df=final_df.append({'month_year': str(item), 'p1': p1_return, 'p2': p2_return, 'p3': p3_return, 'p4': p4_return, 'p5': p5_return, 'p6': p6_return, 'p7': p7_return, 'p8': p8_return, 'p9': p9_return, 'p10': p10_return, 'p11': p11_return, 'p12': p12_return }, ignore_index=True)
      
      
    
   
    
    final_df['average_across_12_portfolios'] = final_df[['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12']].mean(axis=1)
    
    avg_ret = final_df['average_across_12_portfolios'].mean()
    #gmean_ret=scipy.stats.gmean(final_df['average_across_6_portfolios'])
    std_error = np.std(final_df['average_across_12_portfolios'], ddof=1)/np.sqrt(len(final_df))
    avg_ret=avg_ret-1
    t_stat = (avg_ret ) / std_error
  

    table_1_k_12=table_1_k_12.append({'J': j, 'Portfolio': portfolio, 'Return': avg_ret,  't': t_stat}, ignore_index=True)
    print(j)


<ipython-input-21-0777b4e86e84>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_temp['portfolio'] = pd.qcut(crsp_temp[f'{j}_month_return'], q=10, labels=False, duplicates='drop')


3
3
6
6
9
9
12
12


In [ ]:
table_1_k_12

Clean table 1

In [ ]:
table_1_k_3_copy=table_1_k_3
table_1_k_6_copy=table_1_k_6
table_1_k_9_copy=table_1_k_9
table_1_k_12_copy=table_1_k_12


for table, label in zip([table_1_k_3_copy, table_1_k_6_copy, table_1_k_9_copy, table_1_k_12_copy ], [3,6,9,12]):
  table.set_index(['J','Portfolio'], inplace=True)
  table['Return']=table['Return'].round(3)
  table['t']=table['t'].round(3)
  table[f'k={label}'] = table.apply(lambda x: f"{x['Return']} ({x['t']})", axis=1)
  table.drop(['Return', 't'], axis=1, inplace=True)

table1=table_1_k_3_copy.merge(table_1_k_6_copy, left_index=True, right_index=True)
table1=table1.merge(table_1_k_9_copy, left_index=True, right_index=True)
table1=table1.merge(table_1_k_12_copy, left_index=True, right_index=True)

In [32]:
table1=table_1_k_3_copy.merge(table_1_k_6_copy, left_index=True, right_index=True)
table1=table1.merge(table_1_k_9_copy, left_index=True, right_index=True)
table1=table1.merge(table_1_k_12_copy, left_index=True, right_index=True)

In [33]:
table1.reset_index(inplace=True)

In [34]:
table1['Portfolio']=np.where(table1['Portfolio']==0, 'Sell', 'Buy')

In [35]:
table1.set_index(['J', 'Portfolio'], inplace=True)

In [40]:
table1

k=3            k=6            k=9            k=12
J    Portfolio                                                             
3.0  Sell       0.008 (2.411)  0.007 (2.735)  0.006 (3.081)   0.006 (3.432)
     Buy        0.012 (5.475)  0.012 (6.825)  0.012 (8.034)   0.012 (9.077)
6.0  Sell       0.007 (1.842)  0.006 (2.061)  0.005 (2.354)   0.005 (2.755)
     Buy        0.014 (6.412)  0.014 (8.097)  0.013 (9.408)  0.013 (10.396)
9.0  Sell        0.006 (1.72)  0.005 (1.931)  0.005 (2.275)   0.006 (2.713)
     Buy        0.015 (7.113)  0.014 (8.736)  0.014 (9.863)   0.013 (10.74)
12.0 Sell       0.006 (1.675)  0.006 (2.018)  0.006 (2.432)   0.006 (2.911)
     Buy        0.014 (6.716)  0.013 (8.056)  0.012 (9.037)   0.012 (9.788)